In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [3]:
data = pd.read_pickle("data/data_work.pkl")

In [4]:
data["weekday"] = data["weekday"].astype('category')

In [5]:
formula_1 = "price_increase ~ loading_factor + days_since + route"
formula_2 = "price_increase ~ loading_factor + days_since * loading_factor + route"
formula_3 = "price_increase ~ loading_factor + days_since * loading_factor + route + weekday"
formula_4 = "price_increase ~ loading_factor + days_since * loading_factor + route + weekday + peak_hour"
formula_5 = "price_increase ~ loading_factor + days_since * loading_factor + route + weekday + peak_hour + sales_prev_day"
#not used
formula_6 = "price_increase ~ loading_factor + np.power(loading_factor, 2)+ days_since * loading_factor + route + weekday + peak_hour + sales_prev_day"

formulas = [formula_1,formula_2,formula_3,formula_4,formula_5,formula_6]

In [ ]:
data["price_increase"].value_counts()[1]

In [9]:
(data["price_increase"].value_counts()[1]/data["price_increase"].value_counts().sum()).round(4)

0.1163

In [6]:
def logit_estimate(formulas, data, days=[0, 30]):
    output_dict = {}
    for formula in formulas:
        model = smf.logit(
            formula, data=data.loc[data["days_till_dep"].isin(range(days[0], days[1]))]
        )
        output_dict[formula] = model.fit(
            cov_type="cluster",
            cov_kwds={"groups": data.loc[model.data.row_labels, "train_id"]},
        )

    return output_dict

## 30 days

In [23]:
out = logit_estimate(formulas, data, days=[0,33])

Optimization terminated successfully.
         Current function value: 0.334016
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.333892
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.333409
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.331809
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.329979
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.329565
         Iterations 7


In [27]:
out[formulas[4]].get_margeff(at='overall', method='dyex').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:         price_increase
Method:                          dyex
At:                           overall
=============================================================================================
                           dy/d(lnx)    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
route[T.CORDOBAMADRID]        0.0090      0.001      6.293      0.000       0.006       0.012
route[T.MADRIDBARCELONA]      0.0005      0.001      0.332      0.740      -0.002       0.003
route[T.MADRIDCORDOBA]        0.0059      0.001      4.417      0.000       0.003       0.009
route[T.MADRIDSEVILLA]        0.0024      0.001      2.576      0.010       0.001       0.004
route[T.MADRIDZARAGOZA]      -0.0002      0.001     -0.299      0.765      -0.002       0.001
route[T.SEVILLAMADRID]        0.0079      0.001      5.678      0.000       0.005       0.011
route[T.ZARAGOZAMADRID]       0.0012      0.001      1.389      0.165      -0.000       0.003
weekday[T.1]                 -0.0004      0.002     -0.214      0.831      -0.004       0.003
weekday[T.2]                 -0.0022      0.001     -1.792      0.073      -0.005       0.000
weekday[T.3]                 -0.0011      0.001     -1.209      0.227      -0.003       0.001
weekday[T.4]              -5.545e-05      0.001     -0.061      0.951      -0.002       0.002
weekday[T.5]                  0.0009      0.001      0.628      0.530      -0.002       0.004
weekday[T.6]                 -0.0011      0.001     -1.671      0.095      -0.002       0.000
sales_prev_day[T.True]        0.0391      0.004      8.923      0.000       0.031       0.048
loading_factor                0.0798      0.021      3.750      0.000       0.038       0.121
days_since                    0.0250      0.015      1.685      0.092      -0.004       0.054
days_since:loading_factor     0.0435      0.018      2.419      0.016       0.008       0.079
peak_hour                    -0.0085      0.001     -9.543      0.000      -0.010      -0.007
=============================================================================================
"""

In [1]:
out = logit_estimate(formulas, data, days=[0,33])
stargazer = Stargazer(list(out.values()))
HTML(stargazer.render_html())

NameError: name 'logit_estimate' is not defined

In [15]:
out = logit_estimate(formulas, data, days=[0,33])
stargazer = Stargazer(list(out.values()))

stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "loading_factor": "Load factor",
        "days_since": "Day of the booking horizon",
        "peak_hour": "Peak hour",
        "sales_prev_day[T.True]": "Sales experienced on previous day",
        "days_since:loading_factor":"Load factor * day of the BH"
    }
)
stargazer.covariate_order(
    [
        "Intercept",
        "days_since",
        "loading_factor",
        "days_since:loading_factor",
        "peak_hour",
        "sales_prev_day[T.True]",
    ]
)


stargazer.add_line(
    "Route",
    ["Yes", "Yes", "Yes", "Yes", "Yes"],
)
stargazer.add_line(
    "Weekday",
    ["No", "No", "Yes", "Yes", "Yes"],
)
stargazer.add_custom_notes(["Clustered standard errors are in parenthesis"])

output_1 = HTML(stargazer.render_html())

html = output_1.data
with open("regression_outputs/logit_output_30days.html", "w") as f:
    f.write(html)
HTML(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.334016
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.333892
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.333409
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.331809
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.329979
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.329979
         Iterations 7


## last 5 days

In [88]:
data.query("days_till_dep<5")["sales_prev_day"].value_counts()

True     4754
False     448
Name: sales_prev_day, dtype: int64

In [13]:
out = logit_estimate(formulas, data, days=[0,5])

stargazer = Stargazer(list(out.values()))
HTML(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.545135
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.544982
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.541076
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.539536
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.538468
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.537133
         Iterations 6


In [16]:
out[formulas[5]].get_margeff().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:         price_increase
Method:                          dydx
At:                           overall
===============================================================================================
                                 dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
route[T.CORDOBAMADRID]          0.0020      0.029      0.071      0.944      -0.054       0.058
route[T.MADRIDBARCELONA]        0.0193      0.020      0.966      0.334      -0.020       0.058
route[T.MADRIDCORDOBA]         -0.0484      0.034     -1.431      0.153      -0.115       0.018
route[T.MADRIDSEVILLA]          0.0055      0.028      0.200      0.842      -0.049       0.060
route[T.MADRIDZARAGOZA]        -0.0535      0.031     -1.737      0.082      -0.114       0.007
route[T.SEVILLAMADRID]          0.0896      0.026      3.396      0.001       0.038       0.141
route[T.ZARAGOZAMADRID]        -0.0160      0.025     -0.632      0.527      -0.065       0.034
weekday[T.1]                    0.0211      0.022      0.956      0.339      -0.022       0.064
weekday[T.2]                   -0.0099      0.024     -0.410      0.681      -0.057       0.037
weekday[T.3]                   -0.0137      0.029     -0.474      0.635      -0.070       0.043
weekday[T.4]                    0.0399      0.029      1.380      0.168      -0.017       0.097
weekday[T.5]                    0.0737      0.024      3.044      0.002       0.026       0.121
weekday[T.6]                   -0.0704      0.036     -1.946      0.052      -0.141       0.001
sales_prev_day[T.True]          0.0718      0.022      3.193      0.001       0.028       0.116
loading_factor                  1.3838      0.958      1.444      0.149      -0.495       3.262
np.power(loading_factor, 2)    -0.7950      0.243     -3.266      0.001      -1.272      -0.318
days_since                      0.0209      0.021      0.976      0.329      -0.021       0.063
days_since:loading_factor      -0.0035      0.035     -0.099      0.921      -0.073       0.066
peak_hour                      -0.0553      0.017     -3.235      0.001      -0.089      -0.022
===============================================================================================
"""

In [90]:
out = logit_estimate(formulas, data, days=[0,5])

stargazer = Stargazer(list(out.values()))
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "loading_factor": "Load factor",
        "days_since": "Day of the booking horizon",
        "peak_hour": "Peak hour",
        "sales_prev_day[T.True]": "Sales experienced on previous day",
        "days_since:loading_factor":"Load factor * day of the BH"
    }
)
stargazer.covariate_order(
    [
        "Intercept",
        "days_since",
        "loading_factor",
        "days_since:loading_factor",
        "peak_hour",
        "sales_prev_day[T.True]",
    ]
)

stargazer.add_line('Route', ['Yes', 'Yes','Yes','Yes','Yes',])
stargazer.add_line('Weekday', ['No', 'No','Yes','Yes','Yes',])
stargazer.add_custom_notes(["Clustered standard errors are in parenthesis"])
output_1 = HTML(stargazer.render_html())

html = output_1.data
with open("regression_outputs/logit_output_5days.html", "w") as f:
    f.write(html)
HTML(stargazer.render_html())

Optimization terminated successfully.
         Current function value: 0.545135
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.544982
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.541076
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.539536
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.538468
         Iterations 6


30-25 days

In [34]:
out = logit_estimate(formulas, data, days=[5,30])

Optimization terminated successfully.
         Current function value: 0.289634
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.289633
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.289342
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.287715
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.285759
         Iterations 7


In [35]:
stargazer = Stargazer(list(out.values()))
stargazer.rename_covariates({"Intercept": "Constant"})
output_1 = HTML(stargazer.render_html())

html = output_1.data
with open("regression_outputs/logit_output_30days.html", "w") as f:
    f.write(html)
HTML(stargazer.render_html())

## Fixed effect –– not used

In [39]:
from linearmodels import PanelOLS

In [38]:
panel_data = data.set_index(["train_id","days_till_dep"])

In [96]:
fe_lm = PanelOLS.from_formula(
    "price_change_percentage ~ 1 + loading_factor + TimeEffects + EntityEffects",
    panel_data,
    # weights=
).fit(cov_type="clustered", cluster_entity=True)

fe_lm.summary

Dep. Variable:,price_change_percentage,R-squared:,0.0103
Estimator:,PanelOLS,R-squared (Between):,-0.5153
No. Observations:,30336,R-squared (Within):,0.0139
Date:,"Mon, Dec 14 2020",R-squared (Overall):,-0.0029
Time:,15:40:01,Log-likelihood,5.463e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,302.82
Entities:,1209,P-value,0.0000
Avg Obs:,25.092,Distribution:,"F(1,29096)"
Min Obs:,13.000,,
Max Obs:,31.000,F-statistic (robust):,192.32


In [145]:
fe_lm = PanelOLS.from_formula(
    "price_change_percentage ~ 1 + loading_factor + TimeEffects + EntityEffects",
    panel_data.query("days_till_dep >5"),
    # weights=
).fit(cov_type="clustered", cluster_entity=True)

print(fe_lm.summary)

/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


                             PanelOLS Estimation Summary                             
Dep. Variable:     price_change_percentage   R-squared:                        0.0151
Estimator:                        PanelOLS   R-squared (Between):              0.1880
No. Observations:                    23976   R-squared (Within):               0.0008
Date:                     Mon, Dec 14 2020   R-squared (Overall):              0.0121
Time:                             16:55:13   Log-likelihood                 4.436e+04
Cov. Estimator:                  Clustered                                           
                                             F-statistic:                      22.931
Entities:                             1209   P-value                           0.0000
Avg Obs:                            19.831   Distribution:                F(16,23935)
Min Obs:                            8.0000                                           
Max Obs:                            25.000   F-statist

In [60]:
panel_data.columns

Index(['mean_econ_price', 'mean_first_price', 'max_econ_seats',
       'min_econ_seats', 'mean_econ_seats', 'max_first_seats',
       'min_first_seats', 'mean_first_seats', 'departure', 'duration',
       'econ_seats_sold', 'first_seats_sold', 'weekday', 'depart_month',
       'depart_hour', 'day_part', 'price_change', 'price_change_percentage',
       'price_change_direction', 'price_increase', 'seats_sold_prev_day',
       'sales_prev_day', 'max_seat_capacity', 'capacity', 'loading_factor',
       'route', 'peak_hour', 'days_since', 'loading_factor_median',
       'loading_factor_cat'],
      dtype='object')

In [143]:
fe_lm = PanelOLS.from_formula(
    "price_change_percentage ~ 1 + loading_factor + EntityEffects + TimeEffects",
    panel_data.query("days_till_dep <=5"),
).fit(cov_type="clustered", cluster_entity=True)

print(fe_lm.summary)

#F-test for Poolability: 0.9034
#P-value: 0.9867
#ki kéne venni az entity effectet?

/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


                             PanelOLS Estimation Summary                             
Dep. Variable:     price_change_percentage   R-squared:                        0.0128
Estimator:                        PanelOLS   R-squared (Between):             -0.2876
No. Observations:                     6360   R-squared (Within):               0.0210
Date:                     Mon, Dec 14 2020   R-squared (Overall):             -0.0288
Time:                             16:53:41   Log-likelihood                 1.074e+04
Cov. Estimator:                  Clustered                                           
                                             F-statistic:                      66.875
Entities:                             1209   P-value                           0.0000
Avg Obs:                            5.2605   Distribution:                  F(1,5145)
Min Obs:                            1.0000                                           
Max Obs:                            6.0000   F-statist

In [106]:
fe_lm = PanelOLS.from_formula(
    "price_change_percentage ~ 1 + loading_factor + sales_prev_day + peak_hour + weekday + route + TimeEffects",
    panel_data.query("days_till_dep <=5"),
).fit(cov_type="clustered", cluster_entity=True)

fe_lm.summary

/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,price_change_percentage,R-squared:,0.0208
Estimator:,PanelOLS,R-squared (Between):,0.0842
No. Observations:,6360,R-squared (Within):,0.0123
Date:,"Mon, Dec 14 2020",R-squared (Overall):,0.0246
Time:,15:44:33,Log-likelihood,1.018e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8.4208
Entities:,1209,P-value,0.0000
Avg Obs:,5.2605,Distribution:,"F(16,6338)"
Min Obs:,1.0000,,
Max Obs:,6.0000,F-statistic (robust):,10.842


In [134]:
panel_data.query("days_till_dep >=5").reset_index()["days_till_dep"].value_counts()

6     1174
5     1158
13    1150
7     1143
12    1138
10    1136
11    1135
8     1118
9     1115
14    1115
15    1091
16    1091
17    1083
18    1046
19     993
20     962
22     924
21     907
23     896
24     847
25     789
26     724
27     662
28     627
29     583
30     527
Name: days_till_dep, dtype: int64

In [142]:
from linearmodels import FirstDifferenceOLS

fd_panel = FirstDifferenceOLS.from_formula(
    "price_change ~ loading_factor + TimeEffects + EntityEffects",
    panel_data.query("days_till_dep <=5"),
).fit(cov_type="clustered", cluster_entity=True)

fd_panel.summary

/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.8/site-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):


Dep. Variable:,price_change,R-squared:,0.0173
Estimator:,FirstDifferenceOLS,R-squared (Between):,-12.416
No. Observations:,5044,R-squared (Within):,0.0148
Date:,"Mon, Dec 14 2020",R-squared (Overall):,-2.9777
Time:,16:08:57,Log-likelihood,-1.467e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,88.969
Entities:,1209,P-value,0.0000
Avg Obs:,5.2605,Distribution:,"F(1,5043)"
Min Obs:,1.0000,,
Max Obs:,6.0000,F-statistic (robust):,62.637
